In [1]:
from importlib import reload
from src.GEFConstructor import CreateGEF

import matplotlib.pyplot as plt
import numpy as np

In [2]:
#Comparison Run
ntr = 240
tfin = 250#45
tmid = 5#120
GEFComp = CreateGEF("Config/ClassicGEF+beta25+Mass.py")
GEFComp.Unitless()

print("GEFComp loaded")

GEF1 = CreateGEF("Config/ClassicGEF-Test.py")
sol1 = GEF1.RunGEF(ntr, tend=tmid, reachNend=True)

GEFComp loaded
No existing GEF data found. You can specify a file to load using GEF.LoadData or solve the GEF equations using GEF.RunGEF.
The end of inflation was not reached by the solver. Increasing t1 by 58.6 to 63.6.
The end of inflation was not reached by the solver. Increasing t1 by 29.4 to 93.0.


RuntimeError: 

In [ ]:
print(sol1)

  message: A termination event occurred.
  success: True
   status: 1
        t: [ 1.984e+02  1.985e+02 ...  2.225e+02  2.226e+02]
        y: [[ 6.687e+01  6.687e+01 ...  6.724e+01  6.724e+01]
            [ 2.143e-01  2.139e-01 ...  1.438e-01  1.436e-01]
            ...
            [ 1.609e-35  1.021e-35 ...  3.699e-47  2.145e-47]
            [ 1.361e-37  9.106e-38 ...  1.050e-48  6.151e-49]]
      sol: None
 t_events: [array([ 2.226e+02])]
 y_events: [array([[ 6.724e+01,  1.436e-01, ...,  2.024e-47,  5.804e-49]])]
     nfev: 218
     njev: 0
      nlu: 0


NameError: name 'sol2' is not defined

In [ ]:
keys = ["phi", "xi", "H", "dphi", "N"]
Nend1 = GEFComp.EndOfInflation()[0]
Nend2 = GEF1.Nend
for key in keys:
    plt.plot(GEFComp.vals["N"], abs(GEFComp.vals[key]), "gray")
    plt.plot(GEF1.vals["N"], abs(GEF1.vals[key]), "r--")
    plt.vlines(Nend1, min(abs(GEFComp.vals[key])), max(abs(GEFComp.vals[key])), color="orange")
    plt.vlines(Nend2, min(abs(GEFComp.vals[key])), max(abs(GEFComp.vals[key])), color="red", linestyle="dashed")
    plt.show()
keys = ["E", "B", "G", "kh"]
for key in keys:
    plt.plot(GEFComp.vals["N"], abs(GEFComp.vals[key]), "gray")
    plt.plot(GEF1.vals["N"], abs(GEF1.vals[key]), "r--")
    plt.vlines(Nend1, min(abs(GEFComp.vals[key])), max(abs(GEFComp.vals[key])), color="orange")
    plt.vlines(Nend2, min(abs(GEFComp.vals[key])), max(abs(GEFComp.vals[key])), color="red", linestyle="dashed")
    plt.yscale("log")
    plt.show()
        


IndexError: invalid index to scalar variable.

In [ ]:
G.Unitful()

epsH = -G.vals["Hprime"]/G.vals["H"]**2

fig, ax = plt.subplots(3,1, sharex=True, figsize=(8,10))
ax[0].plot(G.vals["N"], G.vals["H"], "k")
ax[0].set_ylim(0.725e-5,1.61e-5)
yticks = np.round(np.arange(0.8, 1.61, 0.2),1)
ytickslab = [rf"${ytick}$" + r"$\times 10^{-5}$" for ytick in yticks]
ax[0].set_yticks(yticks*1e-5, ytickslab)
yticksmin = np.arange(0.75, 1.65, 0.05)
ax[0].set_yticks(yticksmin*1e-5, minor=True)
ax[0].set_ylabel(r"$H/M_{\rm P}$")

ax[1].plot(G.vals["N"], epsH, "k")
yticks = [5e-4, 1e-3, 5e-3, 1e-2, 5e-2, 1e-1]
ytickslab = [r"$5 \times 10^{-4}$", r"$0.001$", r"$0.005$", r"$0.010$", r"$0.050$", r"$0.100$"]

ax[1].set_ylim(4e-4,0.2)
ax[1].set_yscale("log")
ax[1].set_yticks(yticks, ytickslab)
ax[1].set_ylabel(r"$\varepsilon_H$")

ax[2].plot(G.vals["N"], G.vals["xi"], "k")
ax[2].set_ylim(4e-4,0.2)
ax[2].set_yscale("log")

ax[2].set_xticks(np.arange(0, 70, 10))
ax[2].set_xticks(np.arange(0, 70, 2), minor=True)
ax[2].set_xlim(-1, 61)
plt.subplots_adjust(hspace=0)
plt.show()

NameError: name 'G' is not defined